In [48]:
from datetime import datetime
from datetime import timedelta
from sportsreference.ncaab.boxscore import Boxscores
from sportsreference.ncaab.teams import Teams
import pandas as pd
import csv
import os
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [45]:
def get_today():
    today = datetime.today()
    year = today.year
    month = today.month
    day = today.day
    return year,month,day

def get_yesterday():
    today = datetime.today()
    yesterday = today - timedelta(days = 1)
    year = yesterday.year
    month = yesterday.month
    day = yesterday.day
    return year,month,day

def get_games(year, month ,day):
    home_teams = []
    away_teams = []
    games = Boxscores(datetime(year, month, day))
    all_games = games.games[str(month)+'-'+str(day)+'-'+str(year)]
    for game in all_games:
        home_teams.append(game['home_name'])
        away_teams.append(game['away_name'])
        
    name = str(year)+"_"+str(month)+"_"+str(day)+"_games.csv"
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home_team","away_team"])
        for i,team in enumerate(home_teams):
            writer.writerow([home_teams[i],away_teams[i]])
            
    path = os.getcwd()
    os.rename(path+"\\"+name, path+"\\"+"games\\"+name)
    
    return home_teams,away_teams


def get_scores(year, month ,day):
    home_teams = []
    away_teams = []
    home_scores = []
    away_scores = []
    
    games = Boxscores(datetime(year, month, day))
    all_games = games.games[str(month)+'-'+str(day)+'-'+str(year)]
    
    for game in all_games:
        home_teams.append(game['home_name'])
        away_teams.append(game['away_name'])
        home_scores.append(game['home_score'])
        away_scores.append(game['away_score'])
    
    name = str(year)+"_"+str(month)+"_"+str(day)+"_scores.csv"
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home_team","away_team","home_score","away_score"])
        for i,team in enumerate(home_teams):
            writer.writerow([home_teams[i],away_teams[i],home_scores[i],away_scores[i]])
            
    path = os.getcwd()
    os.rename(path+"\\"+name, path+"\\"+"scores\\"+name)       
    return home_teams,away_teams,home_scores,away_scores

def kenpom(year, month ,day):
    options = webdriver.ChromeOptions() 
    #options.add_argument(r'''user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data''')
    #"user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data" home
    # "user-data-dir=C:\Users\koric1\AppData\Local\Google\Chrome\User Data" work
    #options.add_argument("--start-maximized")
    #self.options.add_argument("--headless")
    #self.options.add_argument("--no-sandbox")
    #self.options.add_argument("--disable-gpu")
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')  # Last I checked this was necessa
    driver = webdriver.Chrome(chrome_options= options)


    name = str(year)+"_"+str(month)+"_"+str(day)+"_kenpom.csv"

    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["team","off","def","tempo","luck"])
        #writer.writerow(["SN", "Name", "Contribution"])
        driver.get('https://kenpom.com/')
        time.sleep(1)

        for x in range(1,10):
            try:
                for z in range(1,41):

                    team_name = str(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[2]/a').text)
                    team_off = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[6]').text)
                    team_def = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[8]').text)
                    team_t = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[10]').text)
                    team_luck = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[12]').text)
                    #print(team_name,team_off,team_def,team_t,team_luck)
                    writer.writerow([team_name,team_off,team_def,team_t,team_luck])
            except:
                break
    path = os.getcwd()
    os.rename(path+"\\"+name, path+"\\"+"kenpom\\"+name)
    driver.close()
    

In [46]:
#test get game schedules
home_teams,away_teams = get_games(2020, 2,7)
for i,team in enumerate(home_teams):
    print(home_teams[i], away_teams[i])

FileExistsError: [WinError 183] Cannot create a file when that file already exists: "C:\\Users\\Tarik's PC\\Documents\\college_basketball\\2020_2_7_games.csv" -> "C:\\Users\\Tarik's PC\\Documents\\college_basketball\\games\\2020_2_7_games.csv"

In [49]:
#test kenpom
year,month,day = get_yesterday()
kenpom(year, month,day)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:75: DeprecationWarning: use options instead of chrome_options


Kansas 115.8 85.5 67.3 0.04
Gonzaga 121.3 94.4 71.9 0.05
Baylor 113.5 88.1 66.2 0.016
Dayton 119.1 94.1 67.6 0.002
Duke 115.7 91.1 72.0 -0.009
San Diego St. 115.1 90.6 64.6 -0.008
Michigan St. 115.2 91.2 69.1 -0.012
Ohio St. 114.3 92.0 66.2 -0.038
Louisville 114.5 93.1 67.1 -0.003
West Virginia 108.3 87.1 69.4 -0.028
Maryland 113.5 92.4 67.0 0.037
Creighton 118.2 97.3 68.3 0.072
BYU 116.3 95.6 69.5 -0.028
Houston 112.7 92.3 65.7 -0.052
Florida St. 111.7 91.5 69.9 0.065
Michigan 113.2 93.0 68.7 -0.048
Oregon 117.1 97.1 65.0 0.023
Villanova 113.9 94.0 66.2 0.082
Arizona 111.0 91.4 69.3 -0.077
Seton Hall 112.3 92.7 69.8 0.006
Texas Tech 109.8 90.5 67.3 -0.113
Wisconsin 110.8 91.9 63.7 0.031
Iowa 117.3 98.6 70.2 0.005
Purdue 109.3 90.8 64.6 -0.104
Butler 112.6 94.2 63.7 0.032
Penn St. 110.3 92.4 70.9 0.014
Minnesota 112.1 94.2 66.8 -0.13
Rutgers 107.9 90.2 67.4 -0.042
Kentucky 112.7 95.1 67.7 0.018
Illinois 110.9 93.7 66.0 0.022
Marquette 114.0 96.9 70.7 -0.046
Florida 112.5 95.7 64.9 -0.0

Lipscomb 101.2 108.0 68.7 0.053
New Hampshire 95.0 102.1 68.6 0.011
UT Rio Grande Valley 98.7 106.0 70.4 -0.013
Southern 97.2 104.8 66.8 0.006
Quinnipiac 100.7 108.3 68.9 0.028
Louisiana Monroe 96.1 103.7 64.9 -0.056
Utah Valley 96.5 104.2 70.4 -0.092
Norfolk St. 94.2 101.9 67.5 -0.052
Navy 98.1 106.0 62.5 0.018
UMBC 97.8 105.7 66.8 0.011
VMI 100.2 108.1 67.9 -0.143
Canisius 97.7 105.7 70.0 -0.04
Tennessee St. 96.4 104.6 69.3 0.054
Saint Joseph's 101.9 110.2 71.2 0.009
Purdue Fort Wayne 95.4 103.8 67.9 -0.006
Campbell 96.8 105.3 65.6 -0.047
Cal St. Fullerton 95.9 104.5 69.3 0.01
Army 98.8 107.4 69.5 0.046
Grand Canyon 103.4 112.1 68.0 0.02
Cal St. Bakersfield 96.2 104.9 64.9 -0.063
LIU 97.4 106.1 72.8 0.017
Loyola MD 100.4 109.1 71.0 0.018
Manhattan 92.8 101.6 65.8 -0.011
Northwestern St. 98.3 107.1 72.4 0.015
Milwaukee 96.0 104.9 69.5 -0.015
North Carolina Central 94.2 103.2 66.6 0.065
Weber St. 98.8 107.9 67.6 0.015
Cornell 101.2 110.3 65.2 -0.105
UMass Lowell 104.4 113.6 70.3 -0.036

In [ ]:
#combine for model
def score_kenmpom(year,month,day):
    path = os.getcwd() 
    scores = path+"\\scores\\"str(year)+"_"+str(month)+"_"+str(day)+"_scores.csv"
    kenpom =  path+"\\scores\\"str(year)+"_"+str(month)+"_"+str(day)+"_kenpom.csv"
    
    

In [38]:
year,month,day = get_yesterday()
print(year, month, day)

2020 9 22


In [36]:
type(year)

int

In [37]:
home_teams,away_teams,home_scores,away_scores = get_scores(2020, 2,7)

In [13]:
for i, val in enumerate(home_teams):
    print(home_scores[i],home_teams[i],away_teams[i],away_scores[i])

In [50]:
kp = pd.read_csv('kenpom.csv')
kp.head()

,team,off,def,tempo,luck
0,Kansas,115.8,85.5,67.3,0.040
1,Gonzaga,121.3,94.4,71.9,0.050
2,Baylor,113.5,88.1,66.2,0.016
3,Dayton,119.1,94.1,67.6,0.002
4,Duke,115.7,91.1,72.0,-0.009


In [51]:
for index, row in kp.iterrows():
    name = row['team']
    name_list = name.split()
    if name_list[-1] == "St.":
        name_list[-1] = "State"
        new_name = " ".join(str(x) for x in name_list)
        kp['team'][index] = new_name
        
        
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [52]:
teams_ken_pom = kp['team'].tolist()

In [53]:
teams = Teams()
teams_sportsreference = []
for team in teams:
    teams_sportsreference.append(team.name)

In [54]:
for team in teams_sportsreference:
    if team not in teams_ken_pom:
        print(team)

Alabama-Birmingham
Albany (NY)
Arkansas-Pine Bluff
Bethune-Cookman
Bowling Green State
Brigham Young
Cal State Bakersfield
Cal State Fullerton
Cal State Northridge
California Baptist
UC-Davis
UC-Irvine
UC-Riverside
UC-Santa Barbara
University of California
Central Connecticut State
Central Florida
Citadel
College of Charleston
Detroit Mercy
Florida International
Gardner-Webb
Grambling
Illinois-Chicago
Purdue-Fort Wayne
Cal State Long Beach
Long Island University
Louisiana-Monroe
Louisiana State
Loyola (IL)
Loyola (MD)
Maryland-Baltimore County
Maryland-Eastern Shore
Massachusetts-Lowell
Miami (FL)
Miami (OH)
Missouri-Kansas City
Omaha
Nevada-Las Vegas
North Carolina-Asheville
North Carolina-Greensboro
North Carolina State
North Carolina-Wilmington
Pennsylvania
Prairie View
Saint Francis (PA)
Saint Mary's (CA)
South Carolina Upstate
Southern California
Southern Methodist
Southern Mississippi
St. Francis (NY)
St. John's (NY)
Tennessee-Martin
Texas A&M-Corpus Christi
Texas-Arlington
Texas

In [82]:
map_names_not_same = {
                        "Alabama-Birmingham" : "UAB"
                        "Albany (NY)": "Albany"
                        "Arkansas-Pine Bluff" : "Arkansas Pine Bluff"
                        "Bethune-Cookman" : "Bethune Cookman"
                     }

In [88]:
home_teams,away_teams,home_scores,away_scores = get_scores(2020, 3 ,8)

In [89]:
h_off = []
h_def = []
h_tempo = []
h_luck = []
a_off = []
a_def = []
a_tempo = []
a_luck = []
not_in_h = []
not_in_a = []
for i,team in enumerate(home_teams):
    check = False
    for index, row in kp.iterrows():
        if row['team'] == team:
            h_off.append(row['off'])
            h_def.append(row['def'])
            h_tempo.append(row['tempo'])
            h_luck.append(row['luck'])
            check = True
    if not check:
        h_off.append(None)
        h_def.append(None)
        h_tempo.append(None)
        h_luck.append(None)
        not_in_h.append(i)

for i,team in enumerate(away_teams):
    check = False
    for index, row in kp.iterrows():
        if row['team'] == team and i not in not_in_h:
            a_off.append(row['off'])
            a_def.append(row['def'])
            a_tempo.append(row['tempo'])
            a_luck.append(row['luck'])
            check = True
    if not check:
        not_in_a.append(i)
x = 0
for i in not_in_a:
    num = i - x
    del h_off[num]
    del h_def[num]
    del h_tempo[num]
    del h_luck[num]
    del away_teams[num]
    del home_teams[num]
    x += 1
    

In [90]:
len(h_off)

17

In [91]:
len(home_teams)

17

In [93]:
for i, val in enumerate(home_teams):
    print(home_teams[i],away_teams[i], h_off[i], a_off[i], i + 1)

Maryland Michigan 113.5 113.2 1
Michigan State Ohio State 115.2 114.3 2
Illinois Iowa 110.9 117.3 3
Houston Memphis 112.7 100.6 4
Boston University Bucknell 103.3 96.2 5
Hofstra Drexel 108.7 100.0 6
UCF East Carolina 103.5 99.6 7
William & Mary Elon 103.3 100.6 8
Winthrop Hampton 104.7 104.8 9
Colgate Lafayette 107.2 100.6 10
Liberty Lipscomb 106.1 101.2 11
Minnesota Nebraska 112.1 102.0 12
South Dakota North Dakota 108.6 102.2 13
Towson Northeastern 105.7 105.4 14
Wichita State Tulsa 105.6 103.2 15
Bradley Valparaiso 106.8 104.5 16
Chattanooga Wofford 106.6 104.5 17


In [64]:
for i, val in enumerate(home_teams):
    print(home_scores[i],home_teams[i],away_teams[i],away_scores[i], i)

83 Maryland Michigan 70 0
80 Michigan State Ohio State 69 1
78 Illinois Iowa 76 2
64 Houston Memphis 57 3
64 Boston University Bucknell 61 4
76 Tulane UConn 80 5
67 College of Charleston Delaware 79 6
61 Hofstra Drexel 43 7
94 UCF East Carolina 62 8
63 William & Mary Elon 68 9
76 Winthrop Hampton 68 10
89 Colgate Lafayette 64 11
73 Liberty Lipscomb 57 12
107 Minnesota Nebraska 75 13
79 Oral Roberts Omaha 52 14
71 South Dakota North Dakota 74 15
62 Towson Northeastern 72 16
79 Wichita State Tulsa 57 17
80 Bradley Valparaiso 66 18
97 ETSU Western Carolina 75 19
70 Chattanooga Wofford 72 20
